In [1]:
language = 'evenki'
rus_name = 'Эвенкийский'

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import transformers
transformers.logging.set_verbosity_error()

import numpy as np
import torch

np.random.seed(42)
torch.manual_seed(42)

from transformers import GPT2LMHeadModel, GPT2Tokenizer
tok = GPT2Tokenizer.from_pretrained(f"rugpt_models/weights/{language}_model") #загрузка дообученного токенизатора 

model = GPT2LMHeadModel.from_pretrained(f"rugpt_models/weights/{language}_model") #загрузка дообученной модели

model.cuda()

print('ready')

In [122]:
with open(f'low_resource_mt/datasets/{language}/src-train.txt') as srctrain, open(f'low_resource_mt/datasets/{language}/tgt-train.txt') as tgttrain:
    srctrain = srctrain.readlines()
    tgttrain = tgttrain.readlines()
    train = [(x.strip(), y.strip()) for x,y in zip(srctrain, tgttrain)]

In [123]:
augmented = []

In [124]:
len(train)*0.2

352.0

In [125]:
train_words = set(sum([x.strip().split() for x in srctrain], []))

In [127]:
with open(f'{language}_gpt_augmented.txt', 'w') as file:
    i = len(augmented)
    n = 0
    while len(augmented) < len(train)*0.2:
        n += 1
        inpt = tok.encode(f"{rus_name}:", return_tensors="pt")
        out = model.generate(inpt.cuda(),
                                           num_return_sequences=1,
                                           max_length=40, 
                                           repetition_penalty=1.2,
                                           do_sample=True,
                                           top_k=50, top_p=0.95,
                                           temperature=0.7)
        sent = tok.decode([x.item() if x != 50257 else 0 for x in out[0]]).split('<pad>')[0]
        #print(sent)
        #print('----')
        try:
            w = sent.split(f'{rus_name}: ')[1].split('Перевод')[0].replace('.', ' ').split()
            r = sent.split(f'{rus_name}: ')[1].split('Перевод')[1].replace(':', ' ').split()
            if not set(w).difference(train_words):
                #print('success existence')
                if len(w)/2.5 <= len(r) <= len(w)*2.5:
                    #print('success length')
                    #print('----')
                    #aug_src.append(w)
                    #aug_tgt.append(r)
                    if sent not in augmented:
                        augmented.append(sent)
                        file.write(sent+'\n')
                        i += 1
            if i % 10 == 0:
                print(i, '/', len(train)*0.2, 'total:', n)
        except:
            print('failed')
            if i % 10 == 0:
                print(i, '/', len(train)*0.2, 'total:', n)

10 / 352.0 total: 11
20 / 352.0 total: 23
failed
30 / 352.0 total: 35
40 / 352.0 total: 50
failed
50 / 352.0 total: 65
60 / 352.0 total: 83
60 / 352.0 total: 84
60 / 352.0 total: 85
70 / 352.0 total: 109
failed
80 / 352.0 total: 140
80 / 352.0 total: 141
90 / 352.0 total: 155
failed
90 / 352.0 total: 156
failed
100 / 352.0 total: 181
110 / 352.0 total: 207
110 / 352.0 total: 208
120 / 352.0 total: 225
130 / 352.0 total: 246
130 / 352.0 total: 247
130 / 352.0 total: 248
140 / 352.0 total: 263
140 / 352.0 total: 264
140 / 352.0 total: 265
150 / 352.0 total: 285
150 / 352.0 total: 286
150 / 352.0 total: 287
160 / 352.0 total: 319
160 / 352.0 total: 320
160 / 352.0 total: 321
failed
170 / 352.0 total: 340
170 / 352.0 total: 341
170 / 352.0 total: 342
170 / 352.0 total: 343
170 / 352.0 total: 344
180 / 352.0 total: 369
180 / 352.0 total: 370
180 / 352.0 total: 371
failed
190 / 352.0 total: 396
190 / 352.0 total: 397
190 / 352.0 total: 398
190 / 352.0 total: 399
190 / 352.0 total: 400
190 / 

In [27]:
augmented

['Чукотский: ujŋe. Перевод : нету ',
 'Чукотский: ətrʔes. Перевод : всё ',
 'Чукотский: ətrʔes. Перевод : конец ',
 'Чукотский: ənqene. Перевод : это ',
 'Чукотский: ənqo iwnin. Перевод : потом говорит ',
 'Чукотский: ʔotsoj nəsajoqenat. Перевод: долго сидят молча ',
 'Чукотский: qorəŋe. Перевод : затемнение ',
 'Чукотский: janot nərʔeturi. Перевод с ним расправились ',
 'Чукотский: ew ii mət это. Перевод : ай аж синяк ',
 'Чукотский: mrenti. Перевод : мрэнти ',
 'Чукотский: nəŋʔomore. Перевод : бедствуем мы ',
 'Чукотский: ətrʔes. Перевод: всё ',
 'Чукотский: mrenti. Перевод мрэнти ',
 'Чукотский: niwqin. Перевод : говорит ',
 'Чукотский: tənsikunet. Перевод: dmp обработала ',
 'Чукотский: niwqinet. Перевод: те сказали ',
 'Чукотский: rʔenut. Перевод : я что ',
 'Чукотский: niwqin ətɬʔata. Перевод : мать говорит ',
 'Чукотский: ənqo ilʲja. Перевод : потом илья поймал рыбу ',
 'Чукотский: naqam brendi называлось. Перевод : на самом деле называли бренди ',
 'Чукотский: rʔenut. Перевод :

In [ ]:
train_words = set(sum([x.strip().split() for x in srctrain], []))

In [129]:
aug_src = []
aug_tgt = []

In [130]:
for sent in augmented:
    w = sent.split(f'{rus_name}: ')[1].split('Перевод')[0].replace('.', ' ').split()
    r = sent.split(f'{rus_name}: ')[1].split('Перевод')[1].replace(':', ' ').split()
    #print(set(w).difference(train_words))
    print(len(r), len(w))
    print(sent)
    print('diff words:', set(w).difference(train_words))
    print('length:', len(r), len(w))
    if not set(w).difference(train_words):
        print('success existence')
        if len(w)/2 < len(r) < len(w)*2:
            print('success length')
            aug_src.append(w)
            aug_tgt.append(r)

4 3
Карельский: hiän tuaš vodimah. Перевод: он опять водить будет 
diff words: set()
length: 4 3
success existence
success length
2 2
Карельский: a lutikkoida. Перевод: а клопов 
diff words: set()
length: 2 2
success existence
success length
5 6
Карельский: a mie kuin hänen oboidin značit. Перевод: а я как окажусь значит 
diff words: set()
length: 5 6
success existence
success length
1 1
Карельский: kuin. Перевод: как 
diff words: set()
length: 1 1
success existence
success length
5 5
Карельский: a kuin toko tyttöne pruazniekka. Перевод: а как обычно в праздники 
diff words: set()
length: 5 5
success existence
success length
2 2
Карельский: kuinbuan pruazniekka. Перевод: как праздновали 
diff words: set()
length: 2 2
success existence
success length
1 1
Карельский: konža. Перевод: когда 
diff words: set()
length: 1 1
success existence
success length
6 6
Карельский: kaikki tyttö šielä i vejen lunda. Перевод: все пропали там и братья мои 
diff words: set()
length: 6 6
success existence
s

In [131]:
aug_train = [(" ".join(x), " ".join(y)) for x, y in zip(aug_src, aug_tgt)]

In [132]:
len(train), len(aug_train)

(1760, 340)

In [133]:
new_train = train + aug_train

In [134]:
len(new_train)

2100

In [135]:
import random
random.shuffle(new_train)

In [136]:
import os
if not os.path.exists(f'aug_{language}'):
    os.mkdir(f'aug_{language}')

In [137]:
with open(f'aug_{language}/src-train.txt', 'w') as srctrain, open(f'aug_{language}/tgt-train.txt', 'w') as tgttrain:
    for line1, line2, in new_train:
        srctrain.write(line1+'\n')
        tgttrain.write(line2+'\n')

In [138]:
import shutil
shutil.copy(f'{language}/src-val.txt', f'aug_{language}')
shutil.copy(f'{language}/tgt-val.txt', f'aug_{language}')
shutil.copy(f'{language}/src-test.txt', f'aug_{language}')
shutil.copy(f'{language}/tgt-test.txt', f'aug_{language}')

'aug_karelian/tgt-test.txt'